In [ ]:
##import libraries

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")


In [ ]:
##load dataset

df = pd.read_csv("/Users/xlade/Desktop/Amdari/Internship/Project_2_Jewelry_Price_Optimization/Jewelry_Dataset.csv")
#df.head(5)

In [ ]:
## Assign column names to the dataset

df.columns = [
    "Order_Datetime",
    "Order_ID",
    "Product_ID",
    'SKU_Quantity',
    'Category_ID',
    'Category',
    'Brand_ID',
    "Price_USD",
    "User_ID",
    "Target_Gender",
    "Main_Color", 
    "Main_metal",
    "Main_Gem"
]

In [ ]:
df.head(5)

In [ ]:
## check for missing values
df.isnull().sum()

In [ ]:
#know the data shape

df.shape

In [ ]:
## check fro duplicated rows
df.duplicated().sum()

In [ ]:
## check nunique values
df.nunique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
## drop duplicated rows
df = df.drop_duplicates()


In [ ]:
#check the isnull values again
df.isnull().sum()

In [ ]:
## Look through category feature
df["Category"].unique()

In [ ]:
##filter out the real categories

real_categories = df["Category"].unique().tolist()
real_categories = [c for c in real_categories if isinstance(c, str) and "jewelry" in c ]
real_categories

In [ ]:
df_category = df.loc[df["Category"].isin(real_categories)]
df_category

In [ ]:
## haven filtered out the wrong cats check  isnull again
df_category.isnull().sum()

In [ ]:
## check Target gender
df_category["Target_Gender"].value_counts()

In [ ]:
df_category["Target_Gender"] = df_category["Target_Gender"].fillna(df_category["Target_Gender"].mode()[0])

In [ ]:
df_category.head(5)

In [ ]:
df_category.isnull().sum()

In [ ]:
df_category.shape

In [ ]:
## drop the rest of thr null rows

df_category = df_category.dropna()

In [ ]:
## confirm the isnull is gone
df_category.isnull().sum()

In [ ]:
## confirm the new data size

df_category.shape

In [ ]:
## Observe the unique values again

df_category.nunique()

In [ ]:
df_category.head(5)

## Univariate Analysis

In [ ]:
## category count plot

plt.figure(figsize = (15, 8))

sns.countplot(data = df_category, x = "Category")

plt.xlabel("Jewelry categories")
plt.ylabel("Category frequency")

plt.show(); plt.close()

In [ ]:
## visualize target gender

plt.figure(figsize = (10, 5))

sns.countplot(data = df_category, x = "Target_Gender")

plt.xlabel("Gender")
plt.ylabel("Gender Frequency")

plt.tight_layout()
plt.show()



In [ ]:
## visualize target gender

plt.figure(figsize = (10, 5))

sns.countplot(data = df_category, x = "Main_Color")

plt.xlabel("Main Colour sold")
plt.ylabel("Main colour Frequency")

plt.tight_layout()
plt.show()

In [ ]:
## visualize target gender

plt.figure(figsize = (20, 5))

sns.countplot(data = df_category, x = "Main_Gem")

plt.xlabel("Main Gem sold")
plt.ylabel("Main Gem Frequency")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Bivariate Analysis

In [ ]:
## category count plot

plt.figure(figsize = (15, 8))

sns.countplot(data = df_category, x = "Category", hue="Target_Gender")

plt.xlabel("Jewelry categories")
plt.ylabel("Category frequency")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show(); plt.close()

## Feature Engineering

In [ ]:
## Label Encoding

from sklearn.preprocessing import LabelEncoder

In [ ]:
## Encode the categorical variables

le = LabelEncoder()

df_category["Category"] = le.fit_transform(df_category["Category"])
df_category["Target_Gender"] = le.fit_transform(df_category["Target_Gender"])
df_category["Main_Color"] = le.fit_transform(df_category["Main_Color"])
df_category["Main_metal"] = le.fit_transform(df_category["Main_metal"])
df_category["Main_Gem"] = le.fit_transform(df_category["Main_Gem"])

In [ ]:
##confirm that the features are now encoded

df_category.head(5)

In [ ]:
## feature correlation
correlation = df_category.drop(labels=["Order_Datetime", "SKU_Quantity", "Main_metal"], axis = 1).corr(method = "spearman")

In [ ]:
correlation

In [ ]:
## correlation heatmap

plt.figure(figsize = (10,10))

sns.heatmap(correlation, annot = True, center = .3)

plt.show(); plt.close()

In [ ]:
invariant_column = df_category.nunique()[df_category.nunique() == 1].index.tolist()
invariant_column

In [ ]:
## Drop columns with nunique == 1

df_category.drop(labels=invariant_column, axis = 1, inplace=True)

In [ ]:
columns_to_eliminate = ["Order_Datetime", "Order_ID", "Product_ID", "Category_ID", "User_ID"]

In [ ]:
df_category.drop(labels=columns_to_eliminate, axis = 1, inplace=True)

In [ ]:
df_category.head(5)

In [ ]:
### Seperate label from the rest of the data

df1 = df_category[["Category", "Brand_ID", "Target_Gender", "Main_Color", "Main_Gem"]]
label = df_category[["Price_USD"]]

In [ ]:
df1.head(5)

In [ ]:
label.head(5)

## Machine Learning

In [ ]:
## import 
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from sklearn.impute import SimpleImputer


#Evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# for data preprocessing
from sklearn.model_selection import train_test_split

# Classifier Libraries
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor
from catboost import CatBoostRegressor

from sklearn.preprocessing import PowerTransformer



In [ ]:
import mlflow
import mlflow.sklearn  # Use the relevant MLflow module for your model type
import os

In [ ]:

# Set the tracking URI to the new folder inside Jewelry_Price_Optimization
mlflow.set_tracking_uri("file:///Users/xlade/Desktop/Amdari/Internship/Jewelry_Price_Optimization/mlruns_storage")
mlflow.set_experiment("Test_Experiment")

In [ ]:
# split the dataset into training and test sets x = questions, Y = Answers

X_train, X_test, y_train, y_test = train_test_split(df1, label, test_size = 0.1, random_state = 42)

In [ ]:
# module building

#logistic regression

linreg = LinearRegression()

linreg.fit(X_train, y_train)

lin_y_pred = linreg.predict(X_test)

print("Linear Regression")

print("Mean Absolute Error:", mean_absolute_error(y_test, lin_y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, lin_y_pred))
print("R2 Score:", r2_score(y_test, lin_y_pred))
#print("Accuracy:", accuracy_score(y_test, lin_y_pred))
#print("Precision:", precision_score(y_test, lin_y_pred))
#print("Recall:", recall_score(y_test, lin_y_pred))
#print("F1-score:", f1_score(y_test, lin_y_pred))
#print("AUC-ROC:", roc_auc_score(y_test, lin_y_pred))

In [ ]:
# module building

#Adaboost regression

adaboost = AdaBoostRegressor(loss = "exponential", n_estimators=1000, learning_rate=0.1)

adaboost.fit(X_train, y_train)

adaboost_y_pred = adaboost.predict(X_test)

print("Adaboost Regression")

print("Mean Absolute Error:", mean_absolute_error(y_test, adaboost_y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, adaboost_y_pred))
print("R2 Score:", r2_score(y_test, adaboost_y_pred))


In [ ]:
# module building

#Catboost regression

catboost = CatBoostRegressor(loss_function = "RMSE", iterations=10000, learning_rate=0.1, one_hot_max_size = 2, depth=6, verbose=0)

catboost.fit(X_train, y_train)

catboost_y_pred = catboost.predict(X_test)

print("Catboost Regression")

print("Mean Absolute Error:", mean_absolute_error(y_test, catboost_y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, catboost_y_pred))
print("R2 Score:", r2_score(y_test, catboost_y_pred))


In [ ]:
# module building

#Extra Trees regression

ext = ExtraTreesRegressor(criterion="friedman_mse", n_estimators=1000, bootstrap = True, max_depth=10)

ext.fit(X_train, y_train)

ext_y_pred = ext.predict(X_test)

print("Extra Tree Regressor")

print("Mean Absolute Error:", mean_absolute_error(y_test, ext_y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, ext_y_pred))
print("R2 Score:", r2_score(y_test, ext_y_pred))
